In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
training_data = datasets.FashionMNIST(
    root='/data/yinxiaoln/datasets',
    train=True,
    download=True,
    transform=ToTensor()    
)

test_data = datasets.FashionMNIST(
    root='/data/yinxiaoln/datasets',
    train=False,
    download=True,
    transform=ToTensor()
)

In [8]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape o

In [11]:
device = (
    'cuda' if torch.cuda.is_available()
    else 'mps'
    if torch.backens.mps.is_available()
    else 'cpu'
)

print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>d}/{size:>5d}]')

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f'Test Error: \n Accuracy: {100 * correct:>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [23]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1}\n------------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
------------------------------------
loss: 0.607777 [64/60000]
loss: 0.711893 [6464/60000]
loss: 0.493985 [12864/60000]
loss: 0.736690 [19264/60000]
loss: 0.645343 [25664/60000]
loss: 0.627368 [32064/60000]
loss: 0.680216 [38464/60000]
loss: 0.706565 [44864/60000]
loss: 0.697872 [51264/60000]
loss: 0.644500 [57664/60000]
Test Error: 
 Accuracy: 77.5%, Avg loss: 0.644974

Epoch 2
------------------------------------
loss: 0.588370 [64/60000]
loss: 0.691611 [6464/60000]
loss: 0.477897 [12864/60000]
loss: 0.723657 [19264/60000]
loss: 0.634450 [25664/60000]
loss: 0.617005 [32064/60000]
loss: 0.663142 [38464/60000]
loss: 0.698681 [44864/60000]
loss: 0.687725 [51264/60000]
loss: 0.630200 [57664/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.631713

Epoch 3
------------------------------------
loss: 0.569844 [64/60000]
loss: 0.673807 [6464/60000]
loss: 0.464029 [12864/60000]
loss: 0.711761 [19264/60000]
loss: 0.624584 [25664/60000]
loss: 0.608089 [32064/60000]
loss: 0.647344 [38464

In [24]:
model_path = '/data/yinxiaoln/save/model.pth'
torch.save(model.state_dict(), model_path)
print(f'Saved PyTorch Model State to {model_path}')

Saved PyTorch Model State to /data/yinxiaoln/save/model.pth


In [25]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot'
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to()